In [1]:
working_dir <- "/data/projects/dschaub/ANCA-GN_transcriptomics"
data_dir <- "/data/projects/dschaub/ANCA-GN_transcriptomics/data/single-cell/ustekinumab/integrated_objects"
save_dir <- "/data/projects/dschaub/ANCA-GN_transcriptomics/data/single-cell/ustekinumab"

In [ ]:
RhpcBLASctl::blas_set_num_threads(1)
library(Seurat)
library(dplyr)
# library(tidyverse)
library(here)
# library(readxl)
# library(Matrix)
library(IMSBSeuratHelpers)
library(harmony)
library(ggplot2)
# library(sctransform)
library(future)
library(data.table)
library(xlsx)
plan("multicore", workers = 1)
options(future.globals.maxSize = 20 * 1000 * 1024^2) # 20GB
options(repr.matrix.max.rows = 600, repr.matrix.max.cols = 200)

In [ ]:
path <- paste0(data_dir, "/Harmony_Ustekinumab_4PK4PB_T_celltype.rds")
seu <- readRDS(path)

In [ ]:
DefaultAssay(seu) <- "RNA"

In [ ]:
# DimPlot(object = seu, reduction = 'tsne',label = F,
#        pt.size = 0.1)+ theme(aspect.ratio=1)
DimPlot(
    object = seu, reduction = "umap", label = T,
    pt.size = 0.1
) + theme(aspect.ratio = 1)

In [ ]:
seu <- subset(seu, idents = c("CD8_EM/RM"))
seu

In [ ]:
DimPlot(
    object = seu, reduction = "umap", label = TRUE,
    # cols = cols,
    pt.size = 0.1, label.size = 4, repel = T
) + theme(aspect.ratio = 1, legend.position = "right")

In [ ]:
# str(seu)
# seu@assays$RNA@scale.data

In [ ]:
Idents(seu) <- "patient"
levels(seu)

In [ ]:
seu <- FindVariableFeatures(
    object = seu,
    selection.method = "vst",
    nfeatures = 2000
)

In [ ]:
seu <- ScaleData(seu,
    vars.to.regress = c("nFeature_RNA", "nCount_RNA", "frac.mito") # ,
    # features = rownames(seu)
)
# seu <- ScaleData(seu, features = rownames(seu))
# seu <- ScaleData(seu, features = rownames(seu))

In [ ]:
seu <- RunPCA(
    object = seu,
    features = VariableFeatures(object = seu),
    verbose = T
)

In [ ]:
seu <- seu %>%
    RunHarmony("patient",
        dims.use = 1:30,
        plot_convergence = TRUE
    )

In [ ]:
# seu <- RunTSNE(object = seu,reduction = 'harmony', dims = 1:30)
seu <- RunUMAP(object = seu, reduction = "harmony", dims = 1:30)

In [ ]:
# DimPlot(object = seu, reduction = 'tsne',label = F,
#        pt.size = 0.1)+ theme(aspect.ratio=1)
DimPlot(
    object = seu, reduction = "umap", label = F,
    pt.size = 0.1
) + theme(aspect.ratio = 1)

In [ ]:
seu <- FindNeighbors(object = seu, reduction = "harmony", dims = 1:30)
# !!! Essential step to set the reduction to 'harmony', otherwise the default is PCA and will give big difference of clusters and UMAP
# seu <- FindNeighbors(object = seu, dims = 1:30)

In [ ]:
seu <- FindClusters(object = seu, resolution = 0.75)
table(Idents(seu))
# DimPlot(object = seu, reduction = 'tsne',label = TRUE,
#        pt.size = 0.1,label.size = 6, repel = T) + theme(aspect.ratio=1)
DimPlot(
    object = seu, reduction = "umap", label = TRUE,
    pt.size = 0.1, label.size = 6, repel = T
) + theme(aspect.ratio = 1)

In [ ]:
Idents(seu) <- "tissue"
DimPlot(
    object = seu, reduction = "umap", label = F,
    # split.by = "case",
    pt.size = 0.1
) + theme(aspect.ratio = 1)

In [ ]:
seu <- FindClusters(object = seu, resolution = 0.75)
table(Idents(seu))
# DimPlot(object = seu, reduction = 'tsne',label = TRUE,
#        pt.size = 0.1,label.size = 6, repel = T) + theme(aspect.ratio=1)
DimPlot(
    object = seu, reduction = "umap", label = TRUE,
    pt.size = 0.1, label.size = 6, repel = T
) + theme(aspect.ratio = 1)

In [ ]:
marker_qc <- c("nFeature_RNA", "nCount_RNA", "frac.mito", "frac.ribo", "CD3_count")
for (n in marker_qc) {
    print(
        VlnPlot(
            object = seu, features = n,
            # group.by = "patient",
            # split.by = "cellgroup",
            # x.lab.rot=T,
            # size.x.use = 5,
            pt.size = 0
        ) #+ NoLegend()
    )

    print(FeaturePlot(
        object = seu, features = n,
        cols = c("grey", "blue"),
        order = T,
        reduction = "umap",
        pt.size = 0.1
    ))
}

In [ ]:
DefaultAssay(seu) <- "RNA"

In [ ]:
# find markers for every cluster compared to all remaining cells, report only the positive ones
seu.markers <- FindAllMarkers(
    object = seu, only.pos = TRUE,
    min.pct = 0.25, logfc.threshold = 0.5
)
# logfc.threshold=0.25 (default) instead of old version thresh.use=0.25
head(seu.markers)
dim(seu.markers)

In [ ]:
top3 <- seu.markers %>%
    group_by(cluster) %>%
    top_n(3, avg_log2FC)
top5 <- seu.markers %>%
    group_by(cluster) %>%
    top_n(5, avg_log2FC)
top10 <- seu.markers %>%
    group_by(cluster) %>%
    top_n(10, avg_log2FC)

## DEGs

In [ ]:
options(repr.plot.width = 9, repr.plot.height = 20)
plt1 <- DotPlot(seu,
    features = unique(top10$gene),
    dot.scale = 4
    # scale.by = "size"
) + coord_flip() +
    theme( # strip.background = element_blank(),
        axis.title = element_blank(),
        axis.text.x = element_text(size = 12, angle = 45, hjust = 1), # Increased size from 10 to 12
        axis.text.y = element_text(size = 12), # Increased size from 10 to 12
        legend.position = "right",
        # legend.spacing = unit(0, "mm"),
        legend.direction = "vertical",
        legend.text = element_text(size = 7), # Increased size from 5 to 7
        legend.key.width = unit(2.5, "mm"), # Increased width slightly
        legend.key.height = unit(2.5, "mm"), # Increased height slightly
        legend.box.spacing = unit(1, "mm"),
        legend.margin = margin(2),
        legend.title = element_text(size = 9, angle = 90) # Increased size from 7 to 9
    )
plt1

In [ ]:
DefaultAssay(seu) <- "RNA"
# Search for known marker genes in seu
leukos <- c("PTPRC")
Tcells <- c("CD3G", "CD3D", "CD3E")
CD4 <- c("CD4")
CD8 <- c("CD8A", "CD8B")
Naive <- c("LEF1", "TCF7", "LTB")
CM <- c("CCR7", "SELL", "KLF2", "S1PR1")
RM <- c("CXCR6", "CD69", "ITGAE", "RGS1")
Th1 <- c("CXCR3", "TBX21", "IFNG", "TNF", "CSF2")
Th2 <- c("GATA3", "IL4", "IL5", "IL13")
Th17 <- c("CCR6", "RORC", "IL17A", "IL17F", "IL23R")
Tr1 <- c("IL10", "ITGA2", "LAG3", "HAVCR2") # ,"Ahr","Irf4","Prdm1","Maf")
Tregs <- c("FOXP3", "IL2RA", "CTLA4", "IKZF2", "TIGIT")
Tfh <- c("IL21", "POU2AF1", "CXCR5", "BCL6", "ASCL2", "CD200", "ID3", "ICOS", "ICOSLG")
CTL <- c("PRF1", "GZMB", "GZMK", "GZMA", "GZMH", "GNLY")
NK <- c("NKG7", "KLRC1", "KLRD1", "KLRF1", "KLRB1", "NCR1", "NCAM1", "FGFBP2", "XCL1", "XCL2")
Tgd <- c("TRDV2", "TRGV9")
MAIT <- c("TRAV1-2")
Prolif <- c("STMN1", "MKI67", "TOP2A")

known_markers <- list(
    leukos,
    Tcells,
    CD4,
    CD8,
    CTL,
    NK,
    Naive,
    CM,
    RM,
    Th1,
    Tr1,
    Th2,
    Th17,
    MAIT,
    Tgd,
    Tregs,
    Tfh,
    Prolif
)
known_markers

marker_gene_list <- known_markers
length(unlist(marker_gene_list))
marker_gene_list_expressed <- intersect(unlist(marker_gene_list), rownames(GetAssayData(seu)))
length(marker_gene_list_expressed)
setdiff(unlist(marker_gene_list), marker_gene_list_expressed)

## Yu marker genes

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5)
for (n in marker_gene_list_expressed) {
       print(FeaturePlot(
              object = seu, features = n,
              cols = c("lightgrey", "blue"),
              order = T,
              slot = "data",
              # min.cutoff = "q05", max.cutoff = "q95",
              reduction = "umap",
              pt.size = 0.1
       ))
       print(VlnPlot(
              object = seu, features = n,
              # group.by = "sample",
              # x.lab.rot=T,
              # size.x.use = 5,
              pt.size = 0.01
       ) + NoLegend())
       print(VlnPlot(
              object = seu, features = n,
              # group.by = "sample",
              # x.lab.rot=T,
              # size.x.use = 5,
              pt.size = 0
       ) + NoLegend())
       # print(RidgePlot(seu,
       #          features = n))
}

## Marker proteins Yu

In [ ]:
CITE_list_select <- c(
    "CD3",
    "CD4",
    "CD8",
    "CD45",
    "CD45RA",
    "CCR7",
    "CD62L",
    "CD45RO",
    "CD49a",
    "CD103",
    "CD69",
    "TCRab",
    "TCRgd",
    "TCRVa7.2",
    "TCRVb13.1",
    "TCRVa24Ja18",
    "TCRVd2",
    "TCRVg9",
    "CD44",
    "CTLA4",
    "PDL1",
    "LAG3",
    "ICOS",
    "CD25",
    "TIGIT",
    "CD154",
    "CD161",
    "CD27",
    "CD127",
    "CD169",
    "CD40",
    "CCR4",
    "CXCR3",
    "CCR5",
    "CCR6",
    "CXCR5",
    "CX3CR1",
    "XCR1",
    "ITGB7",
    "IFNGR1",
    "CD106",
    "IL2R",
    "CXCR4",
    "CD2",
    "CD49b",
    "CD28",
    "IL7R",
    "KIR2DL2",
    "KIR3DL1",
    "KIR2DL5",
    "KIR2DL1",
    "NKG2D",
    "NKp30",
    "NKp44",
    "NKp46"
)

In [ ]:
options(repr.plot.width = 9, repr.plot.height = 14)
DefaultAssay(seu) <- "CITE"
plt1 <- DotPlot(seu,
    features = rev(CITE_list_select),
    dot.scale = 2
    # scale.by = "size"
) + coord_flip() +
    theme( # strip.background = element_blank(),
        axis.title = element_blank(),
        axis.text.x = element_text(size = 8, angle = 45, hjust = 1),
        axis.text.y = element_text(size = 6),
        legend.position = "right",
        # legend.spacing = unit(0, "mm"),
        legend.direction = "vertical",
        legend.text = element_text(size = 5),
        legend.key.width = unit(2, "mm"),
        legend.key.height = unit(2, "mm"),
        legend.box.spacing = unit(1, "mm"),
        legend.margin = margin(2),
        legend.title = element_text(size = 7, angle = 90)
    )
plt1

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5)
for (n in CITE_list_select) {
    print(FeaturePlot(
        object = seu, features = n,
        cols = c("lightgrey", "blue"),
        order = T,
        slot = "data",
        # min.cutoff = "q05", max.cutoff = "q95",
        reduction = "umap",
        pt.size = 0.1
    ))
    print(VlnPlot(
        object = seu, features = n,
        # group.by = "sample",
        # x.lab.rot=T,
        # size.x.use = 5,
        pt.size = 0.01
    ) + NoLegend())
    print(VlnPlot(
        object = seu, features = n,
        # group.by = "sample",
        # x.lab.rot=T,
        # size.x.use = 5,
        pt.size = 0
    ) + NoLegend())
    # print(RidgePlot(seu,
    #          features = n))
}

## Convert to h5mu

In [ ]:
library(MuDataSeurat)
library(data.table)

In [ ]:
seu_copy <- copy(seu)

In [ ]:
str(seu_copy)

In [ ]:
seu_copy@assays$RNA@scale.data <- matrix(numeric(), nrow = 0, ncol = 0)
seu_copy@assays$RNA@var.features <- character()
seu_copy@assays$RNA@meta.features <- data.frame(row.names = row.names(seu_copy@assays$RNA@meta.features))
# seu_copy@assays$RNA@key <- "RNA_"
seu_copy@commands <- list()
seu_copy@graphs <- list()
# seu_copy@reductions <- list()
seu_copy@reductions$pca@feature.loadings <- matrix(numeric(), nrow = 0, ncol = 0)
seu_copy@reductions$harmony@feature.loadings <- matrix(numeric(), nrow = 0, ncol = 0)
seu_copy@reductions$harmony@feature.loadings.projected <- matrix(numeric(), nrow = 0, ncol = 0)

In [ ]:
save_dir = paste0(data_dir, "/ANCA_ustekinumab_4PK4PB_CD8Teff.h5mu")
WriteH5MU(seu_copy, save_dir)